In [50]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [54]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

def get_token_length(x):
    tokens = tokenizer.encode(x["text"])
    return {
        "token_length": len(tokens)
    }

get_token_length({"text": "Hello"})

{'token_length': 1}

In [59]:
# dataset_name = "SetFit/emotion"
split = "train"
dataset_name = "yelp_polarity"
dataset = load_dataset(dataset_name, split=split).map(get_token_length)

Found cached dataset yelp_polarity (/Users/casa/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61)


  0%|          | 0/560000 [00:00<?, ?ex/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1144 > 1024). Running this sequence through the model will result in indexing errors


In [60]:
import pandas as pd
df = dataset.to_pandas()
df.head()

,text,label,token_length
0,"Unfortunately, the frustration of being Dr. Go...",0,143
1,Been going to Dr. Goldberg for over 10 years. ...,1,117
2,I don't know what Dr. Goldberg was like before...,0,243
3,I'm writing this review to give you a heads up...,0,238
4,All the food is great here. But the best thing...,1,116


In [61]:
df.token_length.describe()

count    560000.000000
mean        174.207863
std         162.654277
min           1.000000
25%          66.000000
50%         125.000000
75%         227.000000
max        2200.000000
Name: token_length, dtype: float64

In [48]:
df.label.value_counts().sum()

2000

In [42]:
def sample(x):
    if len(x) > 1000:
        return x.sample(n=1000)
    else:
        return x
balanced_emotion = df.groupby("label").apply(sample).reset_index(drop=True)
balanced_emotion.label.value_counts()

0    1000
1    1000
2    1000
3    1000
4    1000
5     572
Name: label, dtype: int64

In [49]:
df.to_json(split + ".json", orient="records", lines=True)